<br>

## __Exercício: Detecção de Anomalias__

<br>

__1:__

Utilizando a classe DetectorAnomalias criada abaixo, __vamos avaliar um detector de anomalias.__

O dataset utilizado pode ser importado através da função getData. 

Nesse conjunto de dados, possuímos 6 variáveis explicativas, $X_1, .., X_6$ e uma variável com a marcação se a instância é uma anomalia ou não.

Utilizando a __metodolodia__ discutida ao longo do módulo, __teste diferentes modelos (variando o limiar $\epsilon$)__ a fim de encontrar o que __melhor fita os dados.__

Justifique as escolhas do $\epsilon$, bem como quais as métricas de performance abordadas. 

<br>

__2:__ 

Aborde o problema num contexto de aprendizado supervisionado, ou seja, treine modelos de classificação binária com o objetivo de detectar anomalias.

Compare os resultados entre as metodologias.

In [1]:
import pandas as pd 
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

In [2]:
class DetectorAnomalias():
    
    def __init__(self, epsilon):
        self.epsilon = epsilon
        
    def fit(self, X):
        medias = X.mean(axis = 0)
        desvios = X.std(axis = 0)
        gaussianas = [st.norm(loc = m, scale = d) for m, d in zip(medias, desvios)]  
        self.gaussianas = gaussianas
        self.X = X
        
    def prob(self, x):
        p = 1
        for i in range(self.X.shape[1]):
            gaussiana_i = self.gaussianas[i]
            x_i = x[i]
            p *= gaussiana_i.pdf(x_i)
        return p
    
    def isAnomaly(self, x):
        return int(np.where(self.prob(x) < self.epsilon, 1, 0))

In [3]:
def getData():
    return pd.read_csv("dataframe_anomalias_exercicio.csv")

In [4]:
df = getData()
df

,x1,x2,x3,x4,x5,x6,anomalia
0,7.731153,23.299155,-0.367453,4.715372,9.306179,16.780965,0.0
1,11.466833,16.943695,-0.245131,7.060311,10.462826,19.821289,0.0
2,11.501272,20.196011,1.206049,-4.957189,7.771262,19.100079,0.0
3,10.893921,16.072385,2.738045,-3.684228,7.373334,23.225524,0.0
4,10.091706,19.253894,0.996895,-9.504052,8.883988,17.903298,0.0
...,...,...,...,...,...,...,...
10095,11.192286,18.451987,-0.953650,-14.362996,10.875826,17.056541,0.0
10096,12.014177,19.461815,1.985099,-7.119190,11.079922,17.582755,0.0
10097,10.745460,18.175951,0.206037,-1.897015,9.888329,17.963324,0.0
10098,9.893969,22.333270,-1.465981,4.137382,7.690620,21.570097,0.0


In [5]:
df.anomalia.value_counts()

0.0    10046
1.0       54
Name: anomalia, dtype: int64

Nosso objetivo é encontrar o melhor limiar para a classe DetectorAnomalias criada acima, para isso iremos seguir um passo a passo, sendo eles:

__1 - Dividir nosso dataset em três datasets da seguinte forma:__

_Dataset de treino com 6000 linhas sem nenhuma anomalia._

_Dataset de validação com 2050 linhas, sendo 27 delas anomalias_

_Dataset de treino com 2050 linhas, sendo 27 delas também anomalias_

__2 - Encontrar a média e desvio padrão de todas as colunas do dataset de treino.__

__3 - Fazer um loop para encontrar a menor gaussiana de todos os registros também no dataset de treino.__


Dessa forma como sabemos que não tem nenhuma anomalia no dataset de treino, podemos escolher o valor encontrado no passo 3 como o limiar para o nosso objeto da classe DetectorAnomalias que criaremos.

Depois disso, podemos comparar esse limiar, com outros limiares da mesma ordem para ver qual deles se sair melhor.

Após encontrar esses limiares, testaremos nosso Detector no dataset de validação e de teste, e por fim, utilizaremos a F1 Score para medir a precisão que temos com os limiares escolhidos.



In [6]:
# separando as anomalias das não anomalias

dfano0 = df[df['anomalia']!=1]
dfano1 = df[df['anomalia']!=0]

In [7]:
dfano0.shape, dfano1.shape

((10046, 7), (54, 7))

In [8]:
# criando os datasets de treino, validacao e teste

dftreino = dfano0.iloc[:6000,:]
dfvalidacao = pd.concat([dfano0.iloc[6000:8023,:],dfano1.iloc[:27,:]],axis = 0)
dfteste =  pd.concat([dfano0.iloc[8023:,:],dfano1.iloc[27:,:]],axis = 0)

In [9]:
dftreino.shape, dfvalidacao.shape, dfteste.shape

((6000, 7), (2050, 7), (2050, 7))

Temos que ter cuidado, pois nosso dataset tem uma coluna "anomália" que não é uma variável explicativa, portanto, vamos criar novos dados de treino, validação e teste, sem a última coluna.

In [10]:
Xtreino = dftreino.values[:,:-1]

Xvalid = dfvalidacao.values[:,:-1]

Xteste = dfteste.values[:,:-1]

Xtreino.shape, Xvalid.shape, Xteste.shape

((6000, 6), (2050, 6), (2050, 6))

Concluimos então o primeiro passo, agora vamos calcular a média e desvio padrão das colunas.

In [11]:
mediaXtreino = Xtreino.mean(axis = 0)
desvioXtreino = Xtreino.std(axis = 0)

Agora criaremos um código onde vamos calcular a função:

$p(x) = p(x_1)\cdot p(x_2) \cdot p(x_3) \cdot p(x_4) \cdot p(x_5) \cdot p(x_6)$, onde cada $p(x_i)$ é a densidade segundo a gaussiana.

Vamos calcular em todas as linhas do dataset de treino e printar o menor valor obtido, assim concluímos o passo 3 e temos um condidato para ser nosso limiar.

In [12]:
listagaussiana = []

for n in range(0,Xtreino.shape[0]): 
    p = 1
    gaussianas = [st.norm(loc = m, scale = d) for m, d in zip(mediaXtreino, desvioXtreino)]
    for i in range(Xtreino.shape[1]):
        gaussiana_i = gaussianas[i]
        x_i = Xtreino[n,][i]
        p*= gaussiana_i.pdf(x_i)
    listagaussiana.append(p)

print('{:.50f}'.format(min(listagaussiana)))
    

0.00000000674761196991165743619185910244680226544034


Obtemos então nosso limiar, vamos construir nosso objeto para detectar anomalias.

In [13]:
dtc = DetectorAnomalias(0.00000000674761196991165743619185910244680226544034)

In [14]:
# fitando nos dados de treino

dtc.fit(Xtreino)

Para conferir, vamos criar uma lista com todas as previsões no próprio dataset de treino, e printar o maior valor, já que não temos anomálias no dataset de treino, esperamos que o valor mostrado seja 0. 

In [15]:
listatreino = []

p = 1
for i in range(Xtreino.shape[0]):
    listatreino.append(dtc.isAnomaly(Xtreino[i, ]))

print(max((listatreino)))

0


Faremos a mesma coisa na lista de validação e de teste, mas dessa vez printaremos a soma dos valores, o melhor valor esperado é de 27 para cada um dos datasets, vamos ver o que conseguimos.

In [16]:
listavalid = []

p = 1
for i in range(Xvalid.shape[0]):
    listavalid.append(dtc.isAnomaly(Xvalid[i, ]))

print(sum((listavalid)))

26


In [17]:
listateste = []

p = 1
for i in range(Xteste.shape[0]):
    listateste.append(dtc.isAnomaly(Xteste[i, ]))

print(sum((listateste)))

24


Não conseguimos uma precisão de 100%, mas chegamos perto, conseguindo obter quase todas as anomalias, tanto nos dados de validação, Vamos calcular o F1 score neles.

In [18]:
from sklearn.metrics import f1_score

In [19]:
ypredvalid = np.array(listavalid).reshape(-1,1)
yvalid = dfvalidacao['anomalia'].values.reshape(-1,1)

ypredteste = np.array(listateste).reshape(-1,1)
yteste = dfteste['anomalia'].values.reshape(-1,1)

In [20]:
f1Scorevalid = f1_score(y_true = yvalid, y_pred = ypredvalid)

f1Scorevalid

0.9811320754716981

In [21]:
f1Scoreteste = f1_score(y_true = yteste, y_pred = ypredteste)

f1Scoreteste

0.9411764705882353

Como imaginavamos, os 98% e 94% concidem com as 26 e 24 anomalias encontradas nos dados de validação e de teste, respectivamente.

Nosso limiar é da cada de $10^{-9}$, vamos então variar um limiar nessa mesma ordem pra ver se conseguimos algum outro limiar com resultado parecido com o obtido.

In [22]:
for limiar in np.arange(0.000000005,0.00000001, 0.000000001):
  
    dc = DetectorAnomalias(limiar)
    dc.fit(Xtreino)
    listatreino = []
    listavalid = []
    listateste = []

    p = 1
    for i in range(Xtreino.shape[0]):
        listatreino.append(dc.isAnomaly(Xtreino[i, ]))


    p = 1
    for i in range(Xvalid.shape[0]):
        listavalid.append(dc.isAnomaly(Xvalid[i, ]))


    p = 1
    for i in range(Xteste.shape[0]):
        listateste.append(dc.isAnomaly(Xteste[i, ]))


    print('\nAnomalias detectadas para Limiar: ', limiar)
    print('\nDados de treino: ', sum((listatreino)))
    print('\nDados de validação: ', sum((listavalid)))
    print('\nDados de teste: ', sum((listateste)))


Anomalias detectadas para Limiar:  5e-09

Dados de treino:  0

Dados de validação:  25

Dados de teste:  16

Anomalias detectadas para Limiar:  6e-09

Dados de treino:  0

Dados de validação:  25

Dados de teste:  23

Anomalias detectadas para Limiar:  7e-09

Dados de treino:  1

Dados de validação:  27

Dados de teste:  25

Anomalias detectadas para Limiar:  7.999999999999999e-09

Dados de treino:  2

Dados de validação:  27

Dados de teste:  28

Anomalias detectadas para Limiar:  9e-09

Dados de treino:  4

Dados de validação:  28

Dados de teste:  29


#### Conseguimos outro limiar também muito bom, $7\cdot 10^{-9}$, que apesar de detectar um elemento dos dados de treino como anomalia, conseguiu encontrar todas as anomalias do dados de validação e quase todas nos dados de teste, vamos calcular o F1 score


In [23]:
dc1 = DetectorAnomalias(0.000000007)

dc1.fit(Xtreino)

In [24]:
listavalid1 = []

p = 1
for i in range(Xvalid.shape[0]):
    listavalid1.append(dc1.isAnomaly(Xvalid[i, ]))

listateste1 = []

p = 1
for i in range(Xteste.shape[0]):
    listateste1.append(dc1.isAnomaly(Xteste[i, ]))
    
ypredvalid1 = np.array(listavalid1).reshape(-1,1)

ypredteste1 = np.array(listateste1).reshape(-1,1)
    
    
f1Scorevalid1 = f1_score(y_true = yvalid, y_pred = ypredvalid1)

f1Scoreteste1 = f1_score(y_true = yteste, y_pred = ypredteste1)

f1Scorevalid1, f1Scoreteste1

(1.0, 0.9615384615384615)

Os dois F1 Score nos diz que realmente todos as anomalias detectadas são as anomalias reais, sem nenhum falso positivo.

Terminamos então o Exercício 1 com dois possíveis Limiares com ótimos resultados:

__0.0000000067 e 0.000000007__

###############################################################################################################################

__2:__ 

Aborde o problema num contexto de aprendizado supervisionado, ou seja, treine modelos de classificação binária com o objetivo de detectar anomalias.

Compare os resultados entre as metodologias.

____________________________________________________________________________________________________

#### Lembrando como é nossa tabela original.

In [25]:
df

,x1,x2,x3,x4,x5,x6,anomalia
0,7.731153,23.299155,-0.367453,4.715372,9.306179,16.780965,0.0
1,11.466833,16.943695,-0.245131,7.060311,10.462826,19.821289,0.0
2,11.501272,20.196011,1.206049,-4.957189,7.771262,19.100079,0.0
3,10.893921,16.072385,2.738045,-3.684228,7.373334,23.225524,0.0
4,10.091706,19.253894,0.996895,-9.504052,8.883988,17.903298,0.0
...,...,...,...,...,...,...,...
10095,11.192286,18.451987,-0.953650,-14.362996,10.875826,17.056541,0.0
10096,12.014177,19.461815,1.985099,-7.119190,11.079922,17.582755,0.0
10097,10.745460,18.175951,0.206037,-1.897015,9.888329,17.963324,0.0
10098,9.893969,22.333270,-1.465981,4.137382,7.690620,21.570097,0.0


Estamos com um problema de classificação, vamos usar dois modelos de Machine Learning, o KNN e o Random Forest.

In [26]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [27]:
dftreino, dfteste = train_test_split(df, test_size = 0.25, random_state = 0)

Vamos começar fazendo uma normalização, com o auxílio da mesma função de pré-processamento usada nos outros módulos.

In [28]:
def preprocessamento_completo(df, dataset_de_treino = True, std_scaler = None):

    dff = df.copy()
    
    variaveis_para_normalizar = ['x1',
                                 'x2',
                                'x3',
                                 'x4',
                                 'x5',
                                 'x6',
                                 ]

    if dataset_de_treino:  

        sc = StandardScaler()
        variaveis_norm = sc.fit_transform(dff[variaveis_para_normalizar])
        
        X, y =  variaveis_norm, dff['anomalia'].values
        return X, y, sc
    
    else:
        
        variaveis_norm = std_scaler.transform(dff[variaveis_para_normalizar]) 
        
        X, y =  variaveis_norm, dff['anomalia'].values
        return X, y

In [29]:
Xtreino, ytreino, stan = preprocessamento_completo(df = dftreino, dataset_de_treino=True, std_scaler=None)

In [30]:
Xteste, yteste = preprocessamento_completo(df =dfteste, dataset_de_treino=False, std_scaler=stan)

In [31]:
Xtreino.shape, ytreino.shape, Xteste.shape, yteste.shape

((7575, 6), (7575,), (2525, 6), (2525,))

Vamos fazer a validação cruzada manualmente.

In [32]:
kf = KFold(n_splits=5)

In [33]:
rf = RandomForestClassifier(n_estimators=1000, max_depth=20)

lista_f1_treino = []
lista_f1_validacao = []

for train_index, val_index in kf.split(Xtreino, ytreino):
    
    Xtrain_folds = Xtreino[train_index]
    ytrain_folds = ytreino[train_index]
    Xval_fold = Xtreino[val_index]
    yval_fold = ytreino[val_index]
    
    rf.fit(Xtrain_folds, ytrain_folds)
    
    pred_treino = rf.predict(Xtrain_folds)
    pred_validacao = rf.predict(Xval_fold)
    
    lista_f1_treino.append(f1_score(y_pred = pred_treino, y_true = ytrain_folds))
    lista_f1_validacao.append(f1_score(y_pred = pred_validacao, y_true = yval_fold))
    
    
print("F1 em treino: \n", lista_f1_treino, " \n| média: ", np.mean(lista_f1_treino))
print()
print("F1 em validação: \n", lista_f1_validacao, " \n| média: ", np.mean(lista_f1_validacao))


F1 em treino: 
 [1.0, 1.0, 1.0, 1.0, 1.0]  
| média:  1.0

F1 em validação: 
 [0.3636363636363636, 0.4, 0.0, 0.0, 0.0]  
| média:  0.15272727272727274


Já conseguimos perceber o que ta acontecendo, temos uma quantidade muito baixa de anomalias, o que deixa muito dificil a previsão correta, vamos agora testar com o KNN, e depois aplicar os dois modelos nos dados de teste.

In [34]:
kn = KNeighborsClassifier(n_neighbors=3)

lista_f1_treino = []
lista_f1_validacao = []

for train_index, val_index in kf.split(Xtreino, ytreino):
    
    Xtrain_folds = Xtreino[train_index]
    ytrain_folds = ytreino[train_index]
    Xval_fold = Xtreino[val_index]
    yval_fold = ytreino[val_index]
    
    kn.fit(Xtrain_folds, ytrain_folds)
    
    pred_treino = kn.predict(Xtrain_folds)
    pred_validacao = kn.predict(Xval_fold)
    
    lista_f1_treino.append(f1_score(y_pred = pred_treino, y_true = ytrain_folds))
    lista_f1_validacao.append(f1_score(y_pred = pred_validacao, y_true = yval_fold))
    
    
print("F1 em treino: \n", lista_f1_treino, " \n| média: ", np.mean(lista_f1_treino))
print()
print("F1 em validação: \n", lista_f1_validacao, " \n| média: ", np.mean(lista_f1_validacao))


F1 em treino: 
 [0.25, 0.1621621621621622, 0.1875, 0.30303030303030304, 0.1875]  
| média:  0.21803849303849304

F1 em validação: 
 [0.0, 0.0, 0.0, 0.0, 0.0]  
| média:  0.0


Como também era de se esperar, modelos de classificação para tratar anomalias não boas opções, para finalizar a comparação, vamos calcular o F1 Score nos dados de teste

In [35]:
f1RF = f1_score(y_pred = rf.predict(Xteste), y_true = yteste)

f1RF

0.19047619047619047

In [36]:
f1KNN = f1_score(y_pred = kn.predict(Xteste), y_true = yteste)

f1KNN

0.0

### Como já comentado, temos uma quantidade muito pequena de anomalias para que modelos de classificação consigam aprender bem sobre eles, por exemplo, se nos dados de treino não tiver nenhuma anomalia, como o modelo irá aprender a achar alguma? Ou Seja, para esse caso, o contexto não supervisionado é o melhor para se trabalhar.